In [1]:
import pandas as pd
import numpy as np
import langdetect 
import re

def data_clean(typee, label):
    def string_clean(s):
        string = re.sub(r'<.*?>','', s) # remove html tag
        string = re.sub(r'http[s]?://\S+', '', string) # remove url
        string = re.sub(r'\\n', ' ', string) # remove new line
        string = re.sub(r'[^a-zA-Z0-9\s]', ' ', string) # remove sepcial characters
        string = re.sub(r'([a-z])([A-Z])', r'\1 \2', string) # separate sticky words
        string = re.sub(r"(?<!\S)[^aA](?!\S)", " ", string)
        words = re.split(r'\s+', string.strip())
        new_string = ' '.join(words)
        return new_string

    file = open(f'./log/data_clean_log/{typee}_repos_with_no_read_me.txt', 'a')
    file2 = open(f'./log/data_clean_log/{typee}_repos_not_english.txt','a')
    df = pd.read_csv(f'{typee}_repos.csv')
    new_df = []
    for row in df.iterrows():
        idx = row[0]
        name = df.iloc[idx]['name']
        stars = df.iloc[idx]['stars']
        url = df.iloc[idx]['url']
        defaultBranch = df.iloc[idx]['defaultBranche']
        des = str(df.iloc[idx]['description'])
        readme = str(df.iloc[idx]['readme'])

        # check if t
        if readme == '[]' or readme=='['']':
            file.write(f'{name},{des},{stars},{url},{defaultBranch},{readme}\n')
            continue

        # check if the repos is English
        try:
            if langdetect.detect(readme) != 'en':
                file2.write(f'{name},{des},{stars},{url},{defaultBranch},{readme}\n')
                continue
        except:
            print(readme)
            continue
        # clean special character for README.md and description
        readme = string_clean(readme)
        des = string_clean(des)

        new_df.append((name, des, stars, url, defaultBranch, readme))

    new_df = pd.DataFrame(data=new_df, columns=['name','description','stars','url',
                                                'defaultBranche','readme'])

    new_df['label'] = label
    new_df.to_csv(f'clean_{typee}_repos.csv', index=False)
    file.close()
    file2.close()

TYPE = ['database','framework','library', 'pl','platform','plugin','toolkit']
for label, t in enumerate(TYPE):
    data_clean(t, label)

['']
['']
['']


In [36]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier

from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline
stopwordsSet = set(stopwords.words("english"))
stopwordsSet.add('use')
stopwordsSet.add('using')
stopwordsSet.add('used')
stopwordsSet.add('user')

def wordcloud_draw(data, color = 'black'):
    words = ' '.join(data)
    cleanedWord = ' '.join([word for word in words.split() if word.lower()  not in stopwordsSet 
                            and len(word) > 1])
    wordcloud = WordCloud(stopwords=STOPWORDS,
                    background_color=color,
                    width=2500,
                    height=2000
                    ).generate(cleanedWord)
    plt.figure(1,figsize=(13, 13))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Minh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
